## Data Extraction


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load input Excel file
input_file_path = "/content/drive/My Drive/Input.xlsx"
df = pd.read_excel(input_file_path)

# Create a directory to save text files if not exists
output_dir = "/content/drive/My Drive/Extracted_Text"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Function to extract article text from URL
def extract_article_text(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        # Find and extract article title
        title = soup.find('title').text.strip()
        # Find and extract article text
        article_text = ''
        for paragraph in soup.find_all('p'):
            article_text += paragraph.text.strip() + '\n'
        return title, article_text
    except Exception as e:
        print(f"Error occurred while processing {url}: {e}")
        return None, None

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']
    # Extract article text from the URL
    title, article_text = extract_article_text(url)
    if title and article_text:
        # Save the extracted text to a text file
        output_file = os.path.join(output_dir, f"{url_id}.txt")
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(f"{title}\n\n{article_text}")
        print(f"Text extracted from {url} and saved to {output_file}")
    else:
        print(f"Failed to extract text from {url}")

print("Extraction completed.")


Mounted at /content/drive
Text extracted from https://insights.blackcoffer.com/rising-it-cities-and-its-impact-on-the-economy-environment-infrastructure-and-city-life-by-the-year-2040-2/ and saved to /content/drive/My Drive/Extracted_Text/blackassign0001.txt
Text extracted from https://insights.blackcoffer.com/rising-it-cities-and-their-impact-on-the-economy-environment-infrastructure-and-city-life-in-future/ and saved to /content/drive/My Drive/Extracted_Text/blackassign0002.txt
Text extracted from https://insights.blackcoffer.com/internet-demands-evolution-communication-impact-and-2035s-alternative-pathways/ and saved to /content/drive/My Drive/Extracted_Text/blackassign0003.txt
Text extracted from https://insights.blackcoffer.com/rise-of-cybercrime-and-its-effect-in-upcoming-future/ and saved to /content/drive/My Drive/Extracted_Text/blackassign0004.txt
Text extracted from https://insights.blackcoffer.com/ott-platform-and-its-impact-on-the-entertainment-industry-in-future/ and saved

## Printing data of extracted file

In [ ]:
import os

# Directory containing the extracted text files
output_dir = "/content/drive/My Drive/Extracted_Text"

# Iterate through each file in the directory
for filename in os.listdir(output_dir):
    if filename.endswith(".txt"):
        filepath = os.path.join(output_dir, filename)
        # Print file name
        print(f"File: {filename}")
        # Open and read the file
        with open(filepath, 'r', encoding='utf-8') as f:
            # Read and print the first 5 lines
            print("Top 5 rows of the file:")
            for i in range(5):
                print(f.readline().strip())
        print("\n---\n")  # Separate outputs for clarity


File: blackassign0001.txt
Top 5 rows of the file:
Rising IT cities and its impact on the economy, environment, infrastructure, and city life by the year 2040. - Blackcoffer Insights

Grafana Dashboard to visualize and analyze sensors’ data
MVP for a software that analyses content from audio (Pharma-based)
Data Engineering and Management tool (Airbyte) with custom data connectors to manage CRM database

---

File: blackassign0002.txt
Top 5 rows of the file:
Rising IT Cities and Their Impact on the Economy, Environment, Infrastructure, and City Life in Future - Blackcoffer Insights

Grafana Dashboard to visualize and analyze sensors’ data
MVP for a software that analyses content from audio (Pharma-based)
Data Engineering and Management tool (Airbyte) with custom data connectors to manage CRM database

---

File: blackassign0003.txt
Top 5 rows of the file:
Internet Demand's Evolution, Communication Impact, and 2035's Alternative Pathways - Blackcoffer Insights

Grafana Dashboard to visual

## Data Anaysis

In [53]:
import os
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from collections import Counter
from textstat import syllable_count
import re
from zipfile import ZipFile

# Load the ZIP file containing the assignment data
zip_file_path = "/content/drive/My Drive/a.zip"

# Extract the content of the ZIP file to a temporary directory
with ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall("/content/tmp")

# Corrected path to the "StopWords" folder
stop_words_folder = "/content/tmp/20211030 Test Assignment/StopWords"

# Load stop words from the "StopWords" folder
stop_words = set()
for file_name in os.listdir(stop_words_folder):
    file_path = os.path.join(stop_words_folder, file_name)
    with open(file_path, 'rb') as f:
        # Try different encodings
        for encoding in ['utf-8', 'latin-1']:  # Add more encodings as needed
            try:
                stop_words.update(f.read().decode(encoding).splitlines())
                break  # Break the loop if decoding succeeds
            except UnicodeDecodeError:
                continue  # Continue to the next encoding if decoding fails

# Load positive and negative words from the "MasterDictionary" folder
master_dict_folder = "/content/tmp/20211030 Test Assignment/MasterDictionary"
positive_words = set()
negative_words = set()
for file_name in os.listdir(master_dict_folder):
    file_path = os.path.join(master_dict_folder, file_name)
    with open(file_path, 'r', encoding='latin-1') as f:
        if file_name.startswith("positive"):
            positive_words.update(f.read().splitlines())
        elif file_name.startswith("negative"):
            negative_words.update(f.read().splitlines())

# Define the output directory for saving analysis results
output_dir = "/content/drive/My Drive/Extracted_Text"

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Function to clean text
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text).lower()  # Remove punctuation and convert to lowercase
    tokens = word_tokenize(text)  # Tokenize text
    filtered_tokens = [word for word in tokens if word not in stop_words]  # Remove stop words
    return filtered_tokens

# Function to compute sentiment scores
def compute_sentiment_scores(tokens):
    positive_score = sum(1 for word in tokens if word in positive_words)
    negative_score = sum(1 for word in tokens if word in negative_words)
    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
    subjectivity_score = (positive_score + negative_score) / ((len(tokens)) + 0.000001)
    return positive_score, negative_score, polarity_score, subjectivity_score

# Function to compute readability analysis
def compute_readability(text):
    sentences = sent_tokenize(text)
    words = word_tokenize(text)
    total_words = len(words)
    total_sentences = len(sentences)
    avg_sentence_length = total_words / total_sentences
    complex_word_count = sum(1 for word in words if syllable_count(word) > 2)
    fog_index = 0.4 * (avg_sentence_length + (complex_word_count / total_words))
    return avg_sentence_length, complex_word_count, fog_index

# Function to compute average number of words per sentence
def compute_avg_words_per_sentence(text):
    sentences = sent_tokenize(text)
    total_words = sum(len(word_tokenize(sentence)) for sentence in sentences)
    total_sentences = len(sentences)
    return total_words / total_sentences

# Function to compute complex word count
def compute_complex_word_count(text):
    words = word_tokenize(text)
    return sum(1 for word in words if syllable_count(word) > 2)

# Function to compute word count
def compute_word_count(text):
    tokens = clean_text(text)
    return len(tokens)

# Function to compute syllable count per word
def compute_syllable_per_word(text):
    words = word_tokenize(text)
    total_syllables = sum(syllable_count(word) for word in words)
    return total_syllables / len(words)

# Function to compute personal pronouns count
def compute_personal_pronouns(text):
    pronouns = re.findall(r'\b(I|we|my|ours|us)\b', text, flags=re.IGNORECASE)
    return len(pronouns)

# Function to compute average word length
def compute_avg_word_length(text):
    words = word_tokenize(text)
    total_chars = sum(len(word) for word in words)
    return total_chars / len(words)

# Create a DataFrame to store the results
output_columns = ['File Name', 'POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE', 'SUBJECTIVITY SCORE',
                  'AVG SENTENCE LENGTH', 'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX',
                  'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT', 'WORD COUNT',
                  'SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH']
output_data = []

# Iterate through each extracted text file
for file_name in os.listdir(output_dir):
    if file_name.endswith(".txt"):
        file_path = os.path.join(output_dir, file_name)
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()

        # Perform text analysis
        cleaned_tokens = clean_text(text)
        sentiment_scores = compute_sentiment_scores(cleaned_tokens)
        readability_scores = compute_readability(text)
        avg_words_per_sentence = compute_avg_words_per_sentence(text)
        complex_word_count = compute_complex_word_count(text)
        word_count = compute_word_count(text)
        syllable_per_word = compute_syllable_per_word(text)
        personal_pronouns = compute_personal_pronouns(text)
        avg_word_length = compute_avg_word_length(text)

        # Append results to output data
        output_data.append((file_name,) + sentiment_scores + readability_scores +
                           (avg_words_per_sentence, complex_word_count, word_count,
                            syllable_per_word, personal_pronouns, avg_word_length))

# Create DataFrame and save to Excel file
output_df = pd.DataFrame(output_data, columns=output_columns)
output_excel_path = "/content/drive/My Drive/Extracted_Text/Output_Data_Structure.xlsx"
output_df.to_excel(output_excel_path, index=False)
print("Data analysis completed and saved to:", output_excel_path)


Data analysis completed and saved to: /content/drive/My Drive/Extracted_Text/Output_Data_Structure.xlsx
